

# Safeguarding your Strands Agents with Amazon Bedrock Guardrails

## Overview
In this example we will guide you through how to create your first Strands Agent using [Amazon Bedrock Guardrails](https://aws.amazon.com/bedrock/guardrails/) to safeguard the underlying Amazon Bedrock LLM model.

Amazon Bedrock Guardrails provides configurable safeguards to help safely build generative AI applications at scale. With a consistent and standard approach used across a wide range of foundation models (FMs) including FMs supported in Amazon Bedrock, fine-tuned models, and models hosted outside of Amazon Bedrock, Guardrails delivers industry-leading safety protections. 

With Strands Agents, you can add Amazon Bedrock Guardrails directly to you Amazon Bedrock models. If you are not using Amazon Bedrock Models, you can use the [Apply Guardrail API](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-use-independent-api.html) to safeguard any model. In this case, you need to build the pipelines using the advance processing capabilities you just learned.


## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|AWS Services used   |Amazon Bedrock Guardrails                          |
|Custom tools created|get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile|
|Agent Structure     |Single agent architecture                          |

</div>


## Architecture

<div style="text-align:left">
    <img src="images/architecture.png" width="85%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with built-in and custom tools
* **Custom tools**: lean how to create your own tools
* **Amazon Bedrock Model as underlying LLM**: Used Anthropic Claude 3.7 from Amazon Bedrock as the underlying LLM model
* **Amazon Bedrock Guardails as undeline safeguard**: Use Amazon Bedrock Guardrails to safe guard your agent application

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock
* IAM role with permissions to create Amazon Bedrock Guardrails

Let's now install the requirement packages for our Strands Agent

In [1]:
!pip install -U -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 199.7 MB/s  0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.38
    Uninstalling botocore-1.37.38:
      Successfully uninstalled botocore-1.37.38
  Attempting uninstall: s3transfer━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [botocore]
    Found existing installation: s3transfer 0.11.3━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
    Uninstalling s3transfer-0.11.3:m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
      Successfully uninstalled s3transfer-0.11.3━━━━━━━━━━━━━━ 2/6 [s3transfer]
  Attempting uninstall: boto30m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
    Found existing installation: boto3 1.37.3━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
    Uninstalling boto3-1.37.3:m━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
      Successfully uninstalled boto3-1.37.3━━━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
  Attempting uninstall: strands-agents━━━━━━━━━━━━━━━━━━━━━━━━ 2/6 [s3transfer]
    Found existing installation: strands-

### Importing dependency packages

Now let's import the dependency packages

In [2]:
import json
import boto3
import os
from strands import Agent, tool
from strands.models import BedrockModel
from customer_profile_tools import get_customer_profile, list_customer_purchases, list_customer_tickets, update_customer_profile
from customer_profiles import CustomerProfileManager, generate_synthetic_profiles

### Generating data

Let's generate some synthetic data for this example. In reality, your agent would connect to an existing customer database, but we will use a JSON file for this example

In [3]:
 # Generate synthetic profiles if needed
profile_manager = CustomerProfileManager()
if not os.path.exists("customer_profiles.json"):
    print("Generating synthetic customer profiles")
    profiles = generate_synthetic_profiles(10)
    print(f"Generated {len(profiles)} synthetic customer profiles")
else:
    print("Using existing customer profiles")
    profiles = list(profile_manager.profiles.values())
    print(f"Loaded {len(profiles)} customer profiles")

# Display sample profile for reference
if profiles:
    sample_profile = profiles[0]
    print(f"\nSample Profile: {sample_profile.name}")
    print(f"Email: {sample_profile.email}")
    print(f"Customer ID: {sample_profile.customer_id}")
    print(f"Location: {sample_profile.state}, {sample_profile.country}")
    print(f"Purchases: {len(sample_profile.purchase_history)}")
    print(f"Support Tickets: {len(sample_profile.support_tickets)}")

Generating synthetic customer profiles
Generated 10 synthetic customer profiles

Sample Profile: Customer 1
Email: customer1@example.com
Customer ID: CUST100
Location: California, USA
Purchases: 1
Support Tickets: 0


## Creating an Amazon Bedrock Guardrail

Let's now create an Amazon Bedrock Guardrail to prevent providing financial advice and filter for common challenges in a customer facing application. 

To create the guardrail, first we need to configure the Amazon Bedrock client:

In [4]:
# Configure AWS clients
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')

You can now use the `create_guardrail` method to create the `no-investment-advice` guardrail that will be used in our application. In this example, our guardrail will include a topic deny for `Fiduciary Advice` that will block our agent to provide any fiduciary advice. The guardrail will also contain some basic content policy to filter innapropriated content and a word policy configuration to detect specific pre-defined words.

In [5]:
# Create a Bedrock guardrail
response = bedrock_client.create_guardrail(
    name='no-investment-advice',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {
                'type': 'PROFANITY'
            }
        ]
    },
    blockedInputMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.',
    blockedOutputsMessaging='I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including financial, or restricted details.',
)

# Print the response to get the guardrail ID
print("Guardrail ID:", response.get('guardrailId'))
print("Guardrail ARN:", response.get('guardrailArn'))

# Store the guardrail ID for later use


Guardrail ID: vvuu0170dcq0
Guardrail ARN: arn:aws:bedrock:us-west-2:120569600315:guardrail/vvuu0170dcq0


In [6]:
guardrail_id = response.get('guardrailId')
guardrail_version = "DRAFT"  # Initial version is always 1

### Testing the guardrail directly

To verify that the guardrail works as expected, we will create the `test_guardrail` support function. this function will use the `apply_guardrail` method to safeguard the input text and provide information about any actions taken by the guardrail 

In [7]:
# Test function to check if input/output is blocked by guardrail
def test_guardrail(text, source_type='INPUT'):
      response = bedrock_runtime.apply_guardrail(
          guardrailIdentifier=guardrail_id,
          guardrailVersion=guardrail_version,
          source=source_type,  # can be 'INPUT' or 'OUTPUT'
          content=[{"text": {"text": text}}]
      )

      # New response format uses different fields
      print(f"Action: {response.get('action')}")
      print(f"Action Reason: {response.get('actionReason', 'None')}")

      # Check if content was blocked
      is_blocked = response.get('action') == 'GUARDRAIL_INTERVENED'
      print(f"Content {source_type} blocked: {is_blocked}")

      if is_blocked:
          # Print topic policies that were triggered
          assessments = response.get('assessments', [])
          if assessments and 'topicPolicy' in assessments[0]:
              print("Blocked topics:", [topic.get('name') for topic in
  assessments[0]['topicPolicy'].get('topics', [])
                                       if topic.get('action') == 'BLOCKED'])

          # Print the modified output if available
          if 'outputs' in response and response['outputs']:
              print("Modified content:", response['outputs'][0].get('text', 'None'))

      return response

# Test some safe input
print("Testing safe input:")
test_guardrail("Tell me about general financial literacy concepts.")

# Test input that should be blocked
print("\nTesting input that should be blocked:")
test_guardrail("What stocks should I invest in for my retirement?")

Testing safe input:
Action: NONE
Action Reason: No action.
Content INPUT blocked: False

Testing input that should be blocked:
Action: GUARDRAIL_INTERVENED
Action Reason: Guardrail blocked.
Content INPUT blocked: True
Blocked topics: ['Fiduciary Advice']
Modified content: I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.


{'ResponseMetadata': {'RequestId': 'ea82c37d-3979-463f-ade6-b4e5f2055f07',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 17 Dec 2025 07:10:10 GMT',
   'content-type': 'application/json',
   'content-length': '1776',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ea82c37d-3979-463f-ade6-b4e5f2055f07'},
  'RetryAttempts': 0},
 'usage': {'topicPolicyUnits': 1,
  'contentPolicyUnits': 1,
  'wordPolicyUnits': 1,
  'sensitiveInformationPolicyUnits': 0,
  'sensitiveInformationPolicyFreeUnits': 0,
  'contextualGroundingPolicyUnits': 0,
  'contentPolicyImageUnits': 0,
  'automatedReasoningPolicyUnits': 0,
  'automatedReasoningPolicies': 0},
 'action': 'GUARDRAIL_INTERVENED',
 'actionReason': 'Guardrail blocked.',
 'outputs': [{'text': 'I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.'}],
 'assessments': [{'topicPolicy': {'topics': [{'name': 

## Integrating with Strands Agent

Now that we confirmed the guardrail is working as expected, let's integrate it the Amazon Bedrock Guardrail with a Strands Agent. This is done via the Bedrock Model object, by setting the `guardrail_id`, `guardrail_version` and `guardrail_trace`. Once the model object is created you can use it to create your agent. We will use a couple of custom tools in this agent: `get_customer_profile`, `list_customer_purchases`, `list_customer_tickets`, `update_customer_profile`. To see their implementation check the `customer_profile_tools.py` file

In [8]:
guardrail_id, guardrail_version

('vvuu0170dcq0', 'DRAFT')

In [9]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,
    guardrail_version=guardrail_version,
    # Enable trace info for debugging
    guardrail_trace="enabled"
)

# Create agent with the guardrail-protected model
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

### Testing the Strands Agent with Guardrails

Let's test our agent with both safe and risky inputs. To do so we will process the agent's response and check if the `stop_reason` is due to a guardrail intervention.

In [10]:
# Helper function to test the agent and check for guardrail interventions
def test_agent_with_guardrail(prompt):
    print(f"\nUser: {prompt}")

    # Get agent response
    response = agent(prompt)

    # Check for guardrail intervention
    if hasattr(response, 'stop_reason') and response.stop_reason == "guardrail_intervened":
        print("\n ⚠️ GUARDRAIL INTERVENED!")
        #print(f"Response: {response}")
    else:
        return response

In [11]:
# Test with a safe question 
test_agent_with_guardrail(
    " what is my latest purchase my Customer ID is CUST100?"
)


User:  what is my latest purchase my Customer ID is CUST100?
I'll help you find information about your latest purchase. Let me look that up using your Customer ID: CUST100.
Tool #1: list_customer_purchases
I apologize, but I wasn't able to find a customer profile with the ID "CUST100" in our system. Could you please verify your Customer ID? If you're unsure about your Customer ID, you might want to provide your email address instead, which I can use to look up your purchase history.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'I apologize, but I wasn\'t able to find a customer profile with the ID "CUST100" in our system. Could you please verify your Customer ID? If you\'re unsure about your Customer ID, you might want to provide your email address instead, which I can use to look up your purchase history.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'list_customer_purchases': ToolMetrics(tool={'toolUseId': 'tooluse_jZC8jueKTBmSqfx-tfWtgA', 'name': 'list_customer_purchases', 'input': {'customer_id': 'CUST100'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005602836608886719)}, cycle_durations=[2.3877995014190674], traces=[<strands.telemetry.metrics.Trace object at 0x7efbe3532990>, <strands.telemetry.metrics.Trace object at 0x7efc184234a0>], accumulated_usage={'inputTokens': 1740, 'outputTokens': 157, 'totalTokens': 1897}, accumulated_metrics={'latencyMs': 4925}), state={}, interrupts=None, str

In [12]:
# Test with a question that asks about investment recommendation, or hate or violence
test_agent_with_guardrail(
    "My SSN is 123-45-6789. Can you help me understand how to protect my retirement account?"
)


User: My SSN is 123-45-6789. Can you help me understand how to protect my retirement account?
I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.
 ⚠️ GUARDRAIL INTERVENED!


## Inspecting Conversation History

Let's examine the conversation history to see how guardrails affected it:

In [13]:
# Print the conversation history
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

Conversation history: [
    {
        "role": "user",
        "content": [
            {
                "text": " what is my latest purchase my Customer ID is CUST100?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'll help you find information about your latest purchase. Let me look that up using your Customer ID: CUST100."
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_jZC8jueKTBmSqfx-tfWtgA",
                    "name": "list_customer_purchases",
                    "input": {
                        "customer_id": "CUST100"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_jZC8jueKTBmSqfx-tfWtgA",
                    "status": "success",
                    "content": [
                      

## Integrating Bedrock guardrails to a chatbot 

Amazon Bedrock provides a built-in guardrails framework that integrates directly with Strands Agents SDK. If a guardrail is triggered, the Strands Agents SDK will automatically overwrite the users input in conversation history. This is done so that follow-up questions are not also blocked by the same questions. This can be configured with the guardrail_redact_input boolean, and the guardrail_redact_input_message string to chage the overwrite message. Additionally, the same functionality is built for the model's output, but this is disabled by default. You can enable this with the guardrail_redact_output boolean, and change the overwrite message with the guardrail_redact_output_message string. Below is an example of how to leverage Bedrock guardrails in your code:

In [14]:
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    guardrail_id=guardrail_id,         # Your Bedrock guardrail ID
    guardrail_version=guardrail_version,             # Guardrail version
    guardrail_trace="enabled",
    guardrail_redact_output = True,        
    guardrail_redact_input = True  ,
    guardrail_redact_input_message = "Guardrail Intervened and Redacted"     
)

# Create agent with the guardrail-protected model
agent = Agent(
    system_prompt="You are a helpful assistant that provides customer support for retail products.",
    model=bedrock_model,
    tools=[
        get_customer_profile,
        list_customer_purchases,
        list_customer_tickets,
        update_customer_profile
    ]
)

In [15]:
# Test a safe question for the customer support Agent
agent(f"what is the data from customer_id {sample_profile.customer_id}?")

I'll help you look up the data associated with customer ID CUST100. Let me retrieve that information for you.
Tool #1: get_customer_profile
I'm sorry, but I couldn't find any customer profile with the ID "CUST100" in our system. There are a few possibilities:

1. The customer ID may have been entered incorrectly
2. The customer might not be registered in our system
3. There might be a different format for customer IDs in our system

Would you like to try searching with a different customer ID or perhaps use an email address instead? That might help us locate the correct customer record.

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'I\'m sorry, but I couldn\'t find any customer profile with the ID "CUST100" in our system. There are a few possibilities:\n\n1. The customer ID may have been entered incorrectly\n2. The customer might not be registered in our system\n3. There might be a different format for customer IDs in our system\n\nWould you like to try searching with a different customer ID or perhaps use an email address instead? That might help us locate the correct customer record.'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'get_customer_profile': ToolMetrics(tool={'toolUseId': 'tooluse_p9MyQFexRJ2RdWS_s8-G4A', 'name': 'get_customer_profile', 'input': {'customer_id': 'CUST100'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004968643188476562)}, cycle_durations=[3.824608325958252], traces=[<strands.telemetry.metrics.Trace object at 0x7efbe0f60b60>, <strands.telemetry.metrics.Trace object at 0x

In [16]:
# Test with user input in which guadrail is intervened and input is redacted with custom message 
agent("How should I allocate my 401(k) investments?")

I apologize, but I am not able to provide fiduciary advice. For your privacy and security, please modify your input and try again without including any financial, or restricted details.

AgentResult(stop_reason='guardrail_intervened', message={'role': 'assistant', 'content': [{'text': '[Assistant output redacted.]'}]}, metrics=EventLoopMetrics(cycle_count=3, tool_metrics={'get_customer_profile': ToolMetrics(tool={'toolUseId': 'tooluse_p9MyQFexRJ2RdWS_s8-G4A', 'name': 'get_customer_profile', 'input': {'customer_id': 'CUST100'}}, call_count=1, success_count=1, error_count=0, total_time=0.0004968643188476562)}, cycle_durations=[3.824608325958252, 0.37746620178222656], traces=[<strands.telemetry.metrics.Trace object at 0x7efbe0f60b60>, <strands.telemetry.metrics.Trace object at 0x7efbf09de090>, <strands.telemetry.metrics.Trace object at 0x7efbe0f61820>], accumulated_usage={'inputTokens': 1737, 'outputTokens': 192, 'totalTokens': 1929}, accumulated_metrics={'latencyMs': 6343}), state={}, interrupts=None, structured_output=None)

The custom message for the redacted input is stored in the conversation history like this :

In [17]:
print(f"Conversation history: {json.dumps(agent.messages, indent=4)}")

Conversation history: [
    {
        "role": "user",
        "content": [
            {
                "text": "what is the data from customer_id CUST100?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "I'll help you look up the data associated with customer ID CUST100. Let me retrieve that information for you."
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_p9MyQFexRJ2RdWS_s8-G4A",
                    "name": "get_customer_profile",
                    "input": {
                        "customer_id": "CUST100"
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_p9MyQFexRJ2RdWS_s8-G4A",
                    "status": "success",
                    "content": [
                        {
             

## Clean up 

Please delete the guardrail as follows :

In [18]:
bedrock_client.delete_guardrail(guardrailIdentifier=guardrail_id)

{'ResponseMetadata': {'RequestId': 'f765f06d-6546-4e35-a96b-789470c8429e',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 17 Dec 2025 07:10:27 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f765f06d-6546-4e35-a96b-789470c8429e'},
  'RetryAttempts': 0}}

## Congratulations!

In this notebook, we demonstrated how to:

1. Create an Amazon Bedrock guardrail that prevents providing financial advice
2. Test the guardrail directly using the Bedrock Runtime API
3. Integrate the guardrail with a Strands agent
4. Test the agent with various inputs to see the guardrail in action
5. Integrate bedrock guardrails in a chatbot with Strands agent.
5. Delete the Guardrail

Guardrails help in keeping AI responses remain safe, compliant, and appropriate for your use case.
